In [ ]:
##### work in this repository was inspired by 
##### https://github.com/Kazuhito00/hand-gesture-recognition-using-mediapipe/blob/main/README_EN.md
##### https://www.youtube.com/watch?v=a99p_fAr6e4


### import dependencies

In [54]:
import csv
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
import pandas as pd


# this is just to make sure that the results are reproducible to anyone that runs the code lol.
RANDOM_SEED = 42

#### data formatting
https://devqa.io/python-convert-csv-file-to-list/

In [45]:
import glob
labels = {0: "start", 1: "turn", 2: "grapple"}

Y_train = [] #1d array, (big_number, 1)
X_train = [] #2d array, (big_number, 42)
def data_append_train(data, X_train = X_train, Y_train = Y_train):
  #formatting the labels, although I am not so sure what is required...
  data1 = "data/train"
  for label in labels: 
    name = labels[label]
    path = f"{data1}/{name}/*.csv"
    print(name)

    # getting the directory
    files = glob.glob(path)
    print(files)
    print(len(files))

    # now concatenating the content of the files.
    for f in files:
        with open(f, 'r') as file:
            csv_reader = csv.reader(file)
        
            i = 0
            for row in csv_reader:
                #skip the header line
                if i == 0: 
                    i += 1
                    continue
                
                X_train.append([float(num) for num in row])
                Y_train.append(label)
    
    print(X_train[-1])
    print("training set:")
    print(len(X_train))
    print(len(Y_train))    

Y_test = []
X_test = []
def data_append_test(data, X_test = X_test, Y_test = Y_test):
  #formatting the labels, although I am not so sure what is required...
  data1 = "data/test"
  for label in labels: 
    name = labels[label]
    path = f"{data1}/{name}/*.csv"
    print(name)

    # getting the directory
    files = glob.glob(path)
    print(files)
    print(len(files))

    # now concatenating the content of the files.
    for f in files:
        with open(f, 'r') as file:
            csv_reader = csv.reader(file)
        
            i = 0
            for row in csv_reader:
                #skip the header line
                if i == 0: 
                    i += 1
                    continue
                
                X_test.append([float(num) for num in row])
                Y_test.append(label)
    
    print(X_test[-1])
    print("training set:")
    print(len(X_test))
    print(len(Y_test))    
#i'll only use the train test.
data = "data"
data_append_train(data)
data_append_test(data)




start
['data/train/start/start26.csv', 'data/train/start/start44.csv', 'data/train/start/start0.csv', 'data/train/start/start1.csv', 'data/train/start/start10.csv', 'data/train/start/start11.csv', 'data/train/start/start12.csv', 'data/train/start/start13.csv', 'data/train/start/start14.csv', 'data/train/start/start15.csv', 'data/train/start/start16.csv', 'data/train/start/start17.csv', 'data/train/start/start18.csv', 'data/train/start/start19.csv', 'data/train/start/start2.csv', 'data/train/start/start20.csv', 'data/train/start/start21.csv', 'data/train/start/start22.csv', 'data/train/start/start23.csv', 'data/train/start/start24.csv', 'data/train/start/start25.csv', 'data/train/start/start27.csv', 'data/train/start/start28.csv', 'data/train/start/start29.csv', 'data/train/start/start3.csv', 'data/train/start/start30.csv', 'data/train/start/start31.csv', 'data/train/start/start32.csv', 'data/train/start/start33.csv', 'data/train/start/start34.csv', 'data/train/start/start35.csv', 'data

### since I already have the training dataset split, I just need to train!

In [50]:
# X_df_train = pd.DataFrame(X_train)
# Y_df_train = pd.DataFrame(Y_train)

# X_df_test = pd.DataFrame(X_test)
# Y_df_test = pd.DataFrame(Y_test)


X_train = np.asarray(X_train)
Y_train = np.asarray(Y_train)
# 
X_test = np.asarray(X_test)
Y_test = np.asarray(Y_test)

from sklearn.utils import shuffle

# the random_state is so that both shuffle perform the same.
X_train,Y_train = shuffle(X_train, Y_train, random_state=0)
X_test,Y_test = shuffle(X_test, Y_test, random_state=0)
# X_train.shape, Y_train.shape
X_train, Y_train


(array([[509.31186676, 459.92548943, 461.01940155, ..., 295.18349648,
         528.78093719, 274.66890335],
        [375.90648651, 256.53282166, 348.931427  , ..., 156.07103348,
         386.70227051, 142.1629715 ],
        [404.26086426, 426.70277596, 352.19688416, ..., 224.50306892,
         444.31091309, 199.56587791],
        ...,
        [ 72.45704651, 233.24028969,  34.21592236, ...,  91.3131237 ,
         129.16862488,  72.83489943],
        [259.07169342, 410.95962524, 228.1410408 , ..., 360.01324654,
         271.52738571, 361.98019981],
        [603.2503891 , 189.52393055, 594.64172363, ..., 168.2949543 ,
         640.63529968, 171.43424034]]),
 array([0, 0, 0, ..., 0, 1, 1]))

In [66]:
NUM_CLASSES = 3
# model_save_path = "model/"
from tensorflow.keras.callbacks import ModelCheckpoint
checkpoint = tf.keras.callbacks.ModelCheckpoint(
    "model/model.{epoch:02d}-{val_accuracy:.2f}",
    monitor='val_accuracy',
    verbose=0,
    save_best_only=True,
    save_weights_only=False,
    mode='auto',
    #everytime the accuracy gets better, it saves
    save_freq='epoch',
)


#defining the model 

In [61]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
import keras

In [71]:

model = Sequential([
    Dense(units=32, activation='relu', input_shape=(42,)),
    Dense(units=64, activation='relu',
        kernel_regularizer=keras.regularizers.l1_l2(0.05)),
    Dense(units=256, activation='relu'),
    # Dropout(0.2),
    Dense(units=128, activation='relu'),
    Dense(units=64, activation='relu'),
    Dense(units=16, activation='relu', 
        kernel_regularizer=keras.regularizers.l1_l2(0.01)),
    # Dropout(0.2),
    Dense(units=NUM_CLASSES, activation='softmax')
])

In [73]:
#cost function
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from tensorflow.keras.optimizers import Adam

model.compile(loss=SparseCategoricalCrossentropy(), optimizer=Adam(), metrics=["accuracy"])

In [74]:
#fitting
history = model.fit(X_train,Y_train, epochs=150, batch_size=32, validation_data=(X_test,Y_test), callbacks=[checkpoint])

Epoch 1/150
77/84 [==========================>...] - ETA: 0s - loss: 19.0000 - accuracy: 0.5901INFO:tensorflow:Assets written to: model/model.01-0.81/assets


INFO:tensorflow:Assets written to: model/model.01-0.81/assets


84/84 [==============================] - 3s 17ms/step - loss: 18.3560 - accuracy: 0.6125 - val_loss: 11.1146 - val_accuracy: 0.8066
Epoch 2/150
82/84 [============================>.] - ETA: 0s - loss: 9.0339 - accuracy: 0.9059INFO:tensorflow:Assets written to: model/model.02-0.89/assets


INFO:tensorflow:Assets written to: model/model.02-0.89/assets


84/84 [==============================] - 1s 13ms/step - loss: 8.9998 - accuracy: 0.9068 - val_loss: 7.7695 - val_accuracy: 0.8896
Epoch 3/150
83/84 [============================>.] - ETA: 0s - loss: 6.5367 - accuracy: 0.9311INFO:tensorflow:Assets written to: model/model.03-0.89/assets


...

84/84 [==============================] - 1s 17ms/step - loss: 0.1391 - accuracy: 0.9888 - val_loss: 0.1959 - val_accuracy: 0.9736
Epoch 78/150
84/84 [==============================] - 0s 4ms/step - loss: 0.1385 - accuracy: 0.9862 - val_loss: 0.3456 - val_accuracy: 0.9075
Epoch 79/150
84/84 [==============================] - 0s 3ms/step - loss: 0.1299 - accuracy: 0.9869 - val_loss: 0.1903 - val_accuracy: 0.9736
Epoch 80/150
84/84 [==============================] - 0s 3ms/step - loss: 0.1079 - accuracy: 0.9940 - val_loss: 0.2564 - val_accuracy: 0.9415
Epoch 81/150
84/84 [==============================] - 0s 3ms/step - loss: 0.1169 - accuracy: 0.9881 - val_loss: 0.2431 - val_accuracy: 0.9255
Epoch 82/150
84/84 [==============================] - 0s 3ms/step - loss: 0.1165 - accuracy: 0.9884 - val_loss: 0.2110 - val_accuracy: 0.9642
Epoch 83/150
84/84 [==============================] - 0s 3ms/step - loss: 0.1336 - accuracy: 0.9855 - val_loss: 0.2452 - val_accuracy: 0.9208
Epoch 84/150
84/84

INFO:tensorflow:Assets written to: model/model.87-0.98/assets


84/84 [==============================] - 1s 18ms/step - loss: 0.0924 - accuracy: 0.9933 - val_loss: 0.1727 - val_accuracy: 0.9755
Epoch 88/150
84/84 [==============================] - 0s 3ms/step - loss: 0.0900 - accuracy: 0.9944 - val_loss: 0.1774 - val_accuracy: 0.9736
Epoch 89/150
84/84 [==============================] - 0s 3ms/step - loss: 0.0941 - accuracy: 0.9914 - val_loss: 0.2520 - val_accuracy: 0.9651
Epoch 90/150
84/84 [==============================] - 0s 3ms/step - loss: 0.0912 - accuracy: 0.9937 - val_loss: 0.2146 - val_accuracy: 0.9274
Epoch 91/150
84/84 [==============================] - 0s 3ms/step - loss: 0.1087 - accuracy: 0.9903 - val_loss: 0.1736 - val_accuracy: 0.9698
Epoch 92/150
84/84 [==============================] - 0s 3ms/step - loss: 0.0788 - accuracy: 0.9937 - val_loss: 0.2120 - val_accuracy: 0.9642
Epoch 93/150
84/84 [==============================] - 0s 3ms/step - loss: 0.0867 - accuracy: 0.9933 - val_loss: 0.2139 - val_accuracy: 0.9340
Epoch 94/150
84/84

INFO:tensorflow:Assets written to: model/model.107-0.98/assets


84/84 [==============================] - 1s 12ms/step - loss: 0.1084 - accuracy: 0.9869 - val_loss: 0.1583 - val_accuracy: 0.9792
Epoch 108/150
84/84 [==============================] - 0s 3ms/step - loss: 0.0953 - accuracy: 0.9914 - val_loss: 0.1799 - val_accuracy: 0.9726
Epoch 109/150
84/84 [==============================] - 0s 3ms/step - loss: 0.0801 - accuracy: 0.9910 - val_loss: 0.2489 - val_accuracy: 0.9642
Epoch 110/150
84/84 [==============================] - 0s 3ms/step - loss: 0.0729 - accuracy: 0.9948 - val_loss: 0.1917 - val_accuracy: 0.9698
Epoch 111/150
84/84 [==============================] - 0s 3ms/step - loss: 0.0877 - accuracy: 0.9903 - val_loss: 0.3180 - val_accuracy: 0.9019
Epoch 112/150
84/84 [==============================] - 0s 3ms/step - loss: 0.0832 - accuracy: 0.9922 - val_loss: 0.2067 - val_accuracy: 0.9698
Epoch 113/150
84/84 [==============================] - 0s 3ms/step - loss: 0.0830 - accuracy: 0.9925 - val_loss: 0.2272 - val_accuracy: 0.9670
Epoch 114/15